In [1]:
import sys
sys.path.append('./source')
from source.code import data_loader 

In [2]:
sys.path

['',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\python36.zip',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Mako-1.0.7-py3.6.egg',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Efim_Golovin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Us